# Binarna klasifikacija korišćenjem neuronskih mreža

Ova sveska uvodi Keras biblioteku i ilustruje njene osnovne koncepte. Zadatak na kojem ćemo raditi je kreiranje neuronske mreže sa propagacijom unapred (engl. feed forward neural network) koja se može iskoristiti u zadatku binarne klasifikacije pacijenata sa dijabetesom.

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
np.random.seed(7)

## Keras biblioteka

[Keras](https://keras.io/) je popularna biblioteka koja nudi podršku u radu sa dubokim neuronskim mrežama. Na njenom razvoju radi Google tim, a zajednica koja je koristi je mnogobrojna. Da bismo mogli da je koristimo u okviru Anaconda okruženja, potrebno ju je instalirati komandom `conda install -c conda-forge keras` u skladu sa [zvaničnim smernicama](https://anaconda.org/conda-forge/keras). Kako je njen API visokog nivoa, u pozadini, na niskom nivou, se može koristiti Tensor Flow, Theano ili CNTK (tzv. backend deo biblioteke). Cilj pobrojanih biblioteka je da pokriju zadatke poput efikasnih algebarskih operacija i numeričog difrerenciranja ili paralelilazija jer rad sa neuronskim mrežama obično podrazumeva i rad sa velikom količinom podataka. U našem radu će to biti Tensor Flow pa je potrebno osigurati se da je i on instaliran. Više o instalaciji Tensor Flow biblioteke možete pročitati [ovde](https://anaconda.org/conda-forge/tensorflow).   

Keras podržava dve vrste API-ja, `standardni` koji ćemo upoznati u primerima koji slede i `funkcionalni` koji daje nešto više fleksibilnosti za nadgradnje biblioteke. Njega ćemo upoznati u priči o konvolutivnim neuronskim mrežama.

In [4]:
import tensorflow as tf
from tensorflow import keras

Za sada ćemo učitati funkcije `Sequential` i `Dense` čije ćemo značenje pojasniti u nastavku.

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

## Skup podataka - Pima Indians Diabetes

Skup podataka "Pima Indians Diabetes" se sastoji od 768 instanci. Svaka instanca sadrži vrednosti 8 medicinski relevantnih vrednosti uz pridruženu klasu sa vrednostima 1 ili 0 u zavisnosti od toga da li je pacijentu u poslednjih 5 godina dijagnostifikovan dijabetes ili ne. 

Svi atributi su numeričkog tipa sa opsegom vrednosti koji varira od atributa do atributa. 

Skup podataka se može preuzeti sa platforme Kaggle sa [ove](https://www.kaggle.com/uciml/pima-indians-diabetes-database#diabetes.csv) adrese.

## Kreiranje neuronske mreže

### Korak 1: Priprema skupa podataka

Kolona `Outcome` ukazuje da li pacijentkinja ima dijabetes (vrednost 1) ili ne (vrednost 0) i nju ćemo koristiti kao ciljnu promenljivu. Preostale kolone predstavljaju različite medicinske podatke i njih ćemo koristiti kao atribute nad kojima će model biti obučavan.

### Korak 2: Kreiranje mreže

Jedan neuron neuronske mreže agregira vrednosti svih signala koji stižu do njega i proizvodi odgovarajući izlazni signal. Signale koje neuron obrađuje nazivamo ulazima neurona (na slici su ulazi obeleženi sa $x_i$). Kako nisu svi signali od iste važnosti, njih u ovoj matematičkoj apstrakciji prate odgovarajuće numeričke težine (na slici su težine obeležene sa $w_i$). Svaki neuron karakteriše i poseban slobodni član (engl. bias term) kojim se utiče na specifična ponašanja neurona (na slici je ovaj član obeležen slovom $b$). Tako, neuron obrađuje težinsku sumu svojih ulaza (na slici obeleženu sa $\Sigma$) uvećanu za vrednost slobodnog člana. Posebno, slobodni član se može posmatrati i kao ulaz neurona kojem je pridružena težina 1 zbog jednostavnijih numeričkih zapisa. Ovako dobijena vrednost se dalje propušta kroz aktivacionu funkciju (na slici obeleženu sa $\phi$) i dobija se izlaz nurona $y$.  

<img src="assets/neuron-math.png" width=550>

Mreža se sastoji od pojedinačnih slojeva neurona. Možemo reći da su slojevi moduli koji obrađuju podatke i da za zadate ulaze generišu odgovarajuće izlaze. Razlikujemo ulazni, prvi, sloj mreže, izlazni, poslednji, sloj mreže i slojeve između koje nazivamo skrivenim slojevima. <img src='assets/layers.png'> Slojeve karakteriše matrica koeficijenata čije optimalne vrednosti mreža u toku treniranja treba da nauči. Samu matricu koeficijenata zovemo `kernel` matricom. Njene vrednosti se mogu nasumično inicijalizovati, ali se u praksi obično koriste neke bolje tehnike inicijalizacije koje dovode do boljeg treniranja mreže. Trenutno najpoznatije su Gloriot (ili Xaviar) i He, obe nazvane po imenima autora. 

Potpuno povezani sloj (engl. fully connected layer), kao što ime kaže, povezuju svaki čvor, neuron, ulaznog sloja sa svakim čvorom, neuronom, izlaznog sloja. Funkcija koju izračunava može se opisati formulom `output = activation(dot(input, kernel) + bias)` gde  `kernel` predstavlja matricu sloja, `bias` vektor njegovih slobodnih članova, `dot` matrično množenje, a `activation` aktivacionu funkciju koja se koristi na nivo sloja. 
<img src='assets/fully_connected_layer_2.png' style='width: 300px'>

Mrežu ćemo napraviti mrežu tako da njena arhitektura zadovoljava sledeće specifikacije:

* **Sloj 1**: potpuno povezani sloj sa 15 neurona i *ReLu* aktivacionom funkcijom 
* **Sloj 2**: potpuno povezani sloj sa 8 neurona i *ReLu* aktivacionom funkcijom
* **Sloj 3**: potpuno povezani sloj sa jednim neuronom i *sigmoidnom* aktivacionom funkcijom


### Korak 3: Treniranje mreže

Da bismo mogli da traniramo mrežu, neophodno je da se opredelimo za funkciju gubitka koja pristaje zadatku i odgovarajući optimizator. Funkcija gubitka (engl. loss function) izračunava koliko dobro mreža predviđa ono što se od nje očekuje i generiše povratni signal neophodan za samo učenje. Optimizator dalje na osnovu ovako dobijenih vrednosti ažurira vrednosti parametara mreže. Cilj treniranja mreže je minimizovati funkciju gubitka uz očuvanje dobre generalizacije. 
<img src='assets/training_steps.png'>

Mrežu ćemo trenirati prema sledećim smernicama:
    
* **Funkcija greške**: binarna unakrsna entropija (engl. binary cross entropy)

* **Optimizator**: Adam

* **Broj epoha**: 50
* **Veličina paketića za treniranje (engl. batch size)**: 32

Na primer, na osnovu prvog grafika mogu se izvesti neke intuicije o optimizaciji, veličini koraka učenja i brzini konvergencije. Sledeća slika sumira zaključke i iskustva.  <img src='assets/loss_monitoring.png'>

### Korak 4: Evaluacija mreže

Da bismo mogli da ocenimo uspešnost treniranja mreže, potrebne su nam vrednosti relevantnih metrika na skupu za testiranje. I u slučaju neuronskih mreža, baš kao i do sada, prilikom evaluacije modela vodimo računa da ni u jednom trenutku ne dođe do preklapanja skupa za treniranje i skupa za testiranje. Evaluacija modela se radi propuštanjem paketića skupa za testiranje kroz mrežu, beleženjem i uprosečavanjem dobijenih vrednosti.

### Korak 5: Korišćenje mreže

Za predikciju vrednosti modela može se koristiti funkcija `predict`.